In [ ]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver
browser = uc.Chrome()

# URL inicial: Página de listados de Madrid
url = "https://www.pisos.com/venta/pisos-madrid/"
browser.get(url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")

# Espera a que los inmuebles se carguen en la página principal
try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
    )
except Exception as e:
    print("TimeoutException: No se encontraron los inmuebles en el tiempo esperado.")
    browser.quit()

# Extraer el primer enlace de la lista de inmuebles
html = browser.page_source
soup = bs(html, 'lxml')

# Encuentra el primer contenedor de inmueble y obtiene el enlace
primer_inmueble = soup.find('div', {'class': 'ad-preview'})
primer_enlace = primer_inmueble.get('data-lnk-href')
primer_enlace_completo = f"https://www.pisos.com{primer_enlace}"

# Accede al primer enlace de inmueble
browser.get(primer_enlace_completo)
time.sleep(3)  # Espera a que la página del primer inmueble cargue

# Lista para almacenar los datos
all_data = []

# Contador de inmuebles visitados
contador_inmuebles = 0
max_inmuebles = 300  # Limitar el scraping a 300 inmuebles

# Bucle para iterar desde el primer inmueble a los siguientes inmuebles desde su página
while contador_inmuebles < max_inmuebles:
    try:
        # Extraer la información del inmueble actual
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Precio
            precio = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}).text.split(' ')[0] if soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}) else "N/A"

            # Superficie construida
            superficie_construida = soup_inmueble.find('span', {'class': 'features__value'}).text.split(' ')[0] if soup_inmueble.find('span', {'class': 'features__value'}) else "N/A"

            # Última actualización
            ultima_actualizacion = soup_inmueble.find('p', {'class': 'last-update__date'}).text if soup_inmueble.find('p', {'class': 'last-update__date'}) else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'}).get_text(strip=True)
                    value = feature.find('span', {'class': 'features__value'}).get_text(strip=True)
                    features_list.append((label, value))

            # Certificado energético
            try:
                energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True)
                emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True)
            except AttributeError:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': browser.current_url,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list
            }

            all_data.append(data)

            # Incrementa el contador de inmuebles
            contador_inmuebles += 1

            # Imprime los datos de la página actual (opcional)
            print(f"Inmueble {contador_inmuebles}: {data}")

        except Exception as e:
            print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Intenta hacer clic en el botón de "Siguiente" para ir al siguiente inmueble
        try:
            siguiente_boton = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation__link.navigation__link--next"))
            )
            siguiente_boton.click()

            # Pausa para esperar a que cargue la siguiente página
            time.sleep(random.uniform(3, 5))

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", termina el bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        break

# Una vez terminado el bucle, puedes convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('pisos_data_300_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


In [6]:
import pandas as pd

# Carga el archivo CSV
file_path = 'pisos_data_limpio_completo.csv'  # Reemplaza con la ruta correcta
df = pd.read_csv(file_path)

# Visualiza las primeras filas
print(df.head())


                                              Enlace  Precio  \
0  https://www.pisos.com/alquilar/piso-la_finca-4...    5.50   
1  https://www.pisos.com/alquilar/casa_unifamilia...    4.00   
2  https://www.pisos.com/alquilar/casa_adosada-se...    3.60   
3  https://www.pisos.com/alquilar/piso-somosaguas...    1.45   
4  https://www.pisos.com/alquilar/piso-valdezarza...  950.00   

   Superficie Construida Última Actualización  Consumo Energético  \
0                  190.0           24/10/2024                15.0   
1                  600.0           24/10/2024                 NaN   
2                  448.0           21/10/2024                 NaN   
3                   69.0           24/10/2024                 NaN   
4                   50.0           24/10/2024                 NaN   

   Emisiones CO2 Tipo de operación  Superficie construida:  Superficie útil:  \
0            2.0          Alquiler                   190.0             170.0   
1            NaN          Alquiler      

In [7]:
df

,Enlace,Precio,Superficie Construida,Última Actualización,Consumo Energético,Emisiones CO2,Tipo de operación,Superficie construida:,Superficie útil:,Habitaciones:,Baños:,Planta:,Antigüedad:,Conservación:,Gastos de comunidad:,Referencia:,Tipo de casa:,Superficie solar:,Exterior:,Interior:
0,https://www.pisos.com/alquilar/piso-la_finca-4...,5.50,190.0,24/10/2024,15.0,2.0,Alquiler,190.0,170.0,3.0,3.0,2ª,Menos de 5 años,En buen estado,Incluidos,2805-002404,Piso,NaN,NaN,NaN
1,https://www.pisos.com/alquilar/casa_unifamilia...,4.00,600.0,24/10/2024,NaN,NaN,Alquiler,600.0,600.0,6.0,7.0,NaN,Entre 5 y 10 años,En buen estado,NaN,2805-002317,Unifamiliar,NaN,NaN,NaN
2,https://www.pisos.com/alquilar/casa_adosada-se...,3.60,448.0,21/10/2024,NaN,NaN,Alquiler,448.0,NaN,5.0,2.0,NaN,Menos de 5 años,En buen estado,NaN,TC170-581681,Adosada,NaN,NaN,NaN
3,https://www.pisos.com/alquilar/piso-somosaguas...,1.45,69.0,24/10/2024,NaN,NaN,Alquiler,69.0,48.0,1.0,1.0,1ª,Menos de 5 años,En buen estado,NaN,2805-002408,Piso,NaN,NaN,NaN
4,https://www.pisos.com/alquilar/piso-valdezarza...,950.00,50.0,24/10/2024,NaN,NaN,Alquiler,50.0,NaN,1.0,1.0,6ª,NaN,NaN,NaN,IF5032-J. Andrés,Piso,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081,https://www.pisos.com/comprar/piso-adelfas2800...,380.00,103.0,15/10/2024,NaN,NaN,venta,103.0,85.0,2.0,1.0,7ª,Entre 10 y 20 años,NaN,NaN,SA515-197V048/1084,Piso,NaN,NaN,NaN
2082,https://www.pisos.com/comprar/chalet_unifamili...,NaN,200.0,23/10/2024,NaN,NaN,venta,200.0,200.0,7.0,5.0,NaN,Entre 30 y 50 años,En buen estado,NaN,1492-424812,Piso,221.0,NaN,NaN
2083,https://www.pisos.com/comprar/chalet-nuevo_ara...,735.00,577.0,01/09/2024,NaN,NaN,venta,577.0,NaN,5.0,4.0,NaN,Entre 20 y 30 años,NaN,NaN,IF6915-CM,Piso,652.0,NaN,NaN
2084,https://www.pisos.com/comprar/casa-colmenar_vi...,622.00,267.0,09/05/2024,NaN,NaN,venta,267.0,230.0,4.0,3.0,NaN,NaN,A estrenar,NaN,994534-CHALET 23- THE VALLEY II,Piso,NaN,NaN,NaN


In [12]:
df["Exterior:"]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
2081    NaN
2082    NaN
2083    NaN
2084    NaN
2085    NaN
Name: Exterior:, Length: 2086, dtype: object

In [14]:
df["Exterior:"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2086 entries, 0 to 2085
Series name: Exterior:
Non-Null Count  Dtype 
--------------  ----- 
69 non-null     object
dtypes: object(1)
memory usage: 16.4+ KB


In [21]:
df["Baños:"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2086 entries, 0 to 2085
Series name: Baños:
Non-Null Count  Dtype  
--------------  -----  
2083 non-null   float64
dtypes: float64(1)
memory usage: 16.4 KB


In [18]:
df["Exterior:"].isnull()

0       True
1       True
2       True
3       True
4       True
        ... 
2081    True
2082    True
2083    True
2084    True
2085    True
Name: Exterior:, Length: 2086, dtype: bool